In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords") 
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from wordcloud import WordCloud, STOPWORDS

In [ ]:
# data = pd.read_csv(r"/kaggle/input/deepnlp/Sheet_1.csv",encoding= "latin1" )

data = pd.read_csv('Sheet_1.csv')
data.drop(["Unnamed: 3","Unnamed: 4","Unnamed: 5",
           "Unnamed: 6","Unnamed: 7",], axis = 1, inplace =True)

data.head()

,response_id,class,response_text
0,response_1,not_flagged,I try and avoid this sort of conflict
1,response_2,flagged,Had a friend open up to me about his mental ad...
2,response_3,flagged,I saved a girl from suicide once. She was goin...
3,response_4,not_flagged,i cant think of one really...i think i may hav...
4,response_5,not_flagged,Only really one friend who doesn't fit into th...


In [ ]:

data.isna().sum()

response_id      0
class            0
response_text    0
dtype: int64

In [ ]:

data = pd.concat([data["class"],data["response_text"]], axis = 1)

# data.dropna(axis=0, inplace =True)
data.head()


,class,response_text
0,not_flagged,I try and avoid this sort of conflict
1,flagged,Had a friend open up to me about his mental ad...
2,flagged,I saved a girl from suicide once. She was goin...
3,not_flagged,i cant think of one really...i think i may hav...
4,not_flagged,Only really one friend who doesn't fit into th...


In [ ]:
embarked_dict = {"not_flagged": 0, "flagged": 1}
data["class"].replace(embarked_dict, inplace=True)

data.head()

,class,response_text
0,0,I try and avoid this sort of conflict
1,1,Had a friend open up to me about his mental ad...
2,1,I saved a girl from suicide once. She was goin...
3,0,i cant think of one really...i think i may hav...
4,0,Only really one friend who doesn't fit into th...


In [ ]:
data.response_text[10]

'Took a week off work, packed up the car and picked up a friend who was on the verge of losing it and went camping/surfing for a week. His parents were a big part of the problem and being away from them and others and physical activity every day for a week. but more just being around helped i feel.'

In [ ]:
first_text = data.response_text[10]
text = re.sub("[^a-zA-Z]"," ",first_text)
text = text.lower() 
print(text)

took a week off work  packed up the car and picked up a friend who was on the verge of losing it and went camping surfing for a week  his parents were a big part of the problem and being away from them and others and physical activity every day for a week  but more just being around helped i feel 


In [ ]:
text = nltk.word_tokenize(text)
text = [ word for word in text if not word in set(stopwords.words("english"))]
print(text)

['took', 'week', 'work', 'packed', 'car', 'picked', 'friend', 'verge', 'losing', 'went', 'camping', 'surfing', 'week', 'parents', 'big', 'part', 'problem', 'away', 'others', 'physical', 'activity', 'every', 'day', 'week', 'around', 'helped', 'feel']


In [ ]:
lemmatizer = WordNetLemmatizer()
text = [(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word, "n"),pos = "v"),pos="a")) for word in text]
print(text)

['take', 'week', 'work', 'pack', 'car', 'pick', 'friend', 'verge', 'lose', 'go', 'camp', 'surf', 'week', 'parent', 'big', 'part', 'problem', 'away', 'others', 'physical', 'activity', 'every', 'day', 'week', 'around', 'help', 'feel']


In [ ]:
description_list = []
for description in data.response_text:
       
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower() 
    
    description = nltk.word_tokenize(description)
    description = [ word for word in description if not word in set(stopwords.words("english"))]
    
    lemmatizer = WordNetLemmatizer()
    description = (lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word, "n"),pos = "v"),pos="a") for word in description)
    
    description = " ".join(description)
    description_list.append(description)


In [ ]:
description_list[10]

'take week work pack car pick friend verge lose go camp surf week parent big part problem away others physical activity every day week around help feel'

In [ ]:
max_features = 100
count_vectorizer = CountVectorizer(max_features=max_features)
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("Top {} Most Used Words: {}".format(max_features,count_vectorizer.get_feature_names_out()))

Top 100 Most Used Words: ['addiction' 'advice' 'alone' 'always' 'anxiety' 'anything' 'back' 'best'
 'bring' 'call' 'care' 'come' 'comfort' 'could' 'deal' 'depression'
 'describe' 'dont' 'end' 'even' 'everything' 'experience' 'face' 'feel'
 'find' 'friend' 'get' 'gf' 'girl' 'girlfriend' 'give' 'go' 'good' 'grade'
 'happen' 'help' 'helpful' 'issue' 'kid' 'kill' 'know' 'last' 'let' 'life'
 'like' 'listen' 'little' 'look' 'lot' 'make' 'many' 'may' 'much' 'need'
 'never' 'night' 'offer' 'often' 'one' 'open' 'others' 'people' 'person'
 'personal' 'pretty' 'problem' 'really' 'relationship' 'say' 'school'
 'see' 'self' 'severe' 'share' 'shit' 'similar' 'simply' 'situation'
 'someone' 'sometimes' 'start' 'struggle' 'stuff' 'suicide' 'support'
 'talk' 'tell' 'think' 'though' 'time' 'trouble' 'try' 'use' 'want' 'way'
 'week' 'well' 'work' 'would' 'year']


In [ ]:
y = data.iloc[:,0].values
x = sparce_matrix
print(x)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [ ]:
nb = GaussianNB()
nb.fit(x_train,y_train)

GaussianNB()

In [ ]:

y_pred = nb.predict(x_test)
# print("Accuracy: {}".format(round(nb.score(y_pred.reshape(-1,1),y_test),2)))
# print(y_pred)

In [ ]:

acc_score = accuracy_score(y_test, y_pred)
conf_mat = confusion_matrix(
        y_test, y_pred)

In [ ]:
print(acc_score)
print(conf_mat)

0.75
[[12  2]
 [ 2  0]]


In [ ]:
#verifying for sheet-2
vdata = pd.read_csv('Sheet_2.csv')
vdata.drop(["Unnamed: 3","Unnamed: 4","Unnamed: 5",
           "Unnamed: 6","Unnamed: 7",], axis = 1, inplace =True)

vdata.head()